## 컨텐츠 기반 추천 - 영화 추천 Ver.2
- 영화 줄거리, 감독, 주연배우 --> Tfidf 변환 --> 코사인 유사도

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
movie = pd.read_csv('data/movies_metadata.csv', low_memory=False)
info = pd.read_csv('data/credits.csv')
info.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


#### 1. 데이터 전처리 - movie와 info 데이터 병합

In [3]:
df = movie[['id', 'title', 'overview']]
df.head(3)

,id,title,overview
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...


In [4]:
# 결측치 제거, 중복 제거
df.dropna(how='any', inplace=True)
df.drop_duplicates(subset=['overview'], inplace=True)
df.drop_duplicates(subset=['title'], inplace=True)
df.shape

(41218, 3)

In [5]:
type(df.id[0])

str

In [6]:
df.id = df.id.astype(int)
info.id = info.id.astype(int)
df = pd.merge(df, info)
df.head(2)

,id,title,overview,cast,crew
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


In [7]:
df.set_index('id', inplace=True)
df.head(2)

,title,overview,cast,crew
id,,,,
862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


- cast column 처리

In [8]:
df.cast[862]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [9]:
from ast import literal_eval
df.cast = df.cast.apply(literal_eval)
df.head(2)

,title,overview,cast,crew
id,,,,
862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


In [10]:
df.cast[862][0]['name']

'Tom Hanks'

In [11]:
len(df.cast[862])

13

In [12]:
# 3명의 주인공 꺼내기
cast = []
for item in df.cast[862]:
    if item['order'] < 3:
        cast.append(item['name'])
cast

['Tom Hanks', 'Tim Allen', 'Don Rickles']

In [13]:
# cast list에서 주연 배우들의 공백을 지우고 문자열로 만들기
cast = map(lambda s: s.replace(' ', ''), cast)
' '.join(cast)

'TomHanks TimAllen DonRickles'

In [14]:
# 위의 두 과정을 함수로 만들기
def get3actors(x):
    cast = []
    for item in x:
        if item['order'] < 3:
            cast.append(item['name'])
    cast = map(lambda s: s.replace(' ', ''), cast)
    return ' '.join(cast)

In [15]:
df['cast3'] = df.cast.apply(get3actors)
df.head(3)

,title,overview,cast,crew,cast3
id,,,,,
862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",TomHanks TimAllen DonRickles
8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",RobinWilliams JonathanHyde KirstenDunst
15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",WalterMatthau JackLemmon Ann-Margret


- 감독 처리하기

In [18]:
df.crew = df.crew.apply(literal_eval)
df.crew[862][:2]

[{'credit_id': '52fe4284c3a36847f8024f49',
  'department': 'Directing',
  'gender': 2,
  'id': 7879,
  'job': 'Director',
  'name': 'John Lasseter',
  'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f4f',
  'department': 'Writing',
  'gender': 2,
  'id': 12891,
  'job': 'Screenplay',
  'name': 'Joss Whedon',
  'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'}]

In [19]:
def get_director(x):
    for item in x:
        if item['job'] == 'Director':
            return item['name'].replace(' ', '')
    return ''

In [20]:
df['director'] = df.crew.apply(get_director)
df.head(3)

,title,overview,cast,crew,cast3,director
id,,,,,,
862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",TomHanks TimAllen DonRickles,JohnLasseter
8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",RobinWilliams JonathanHyde KirstenDunst,JoeJohnston
15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",WalterMatthau JackLemmon Ann-Margret,HowardDeutch


#### 2. Feature 변환

In [21]:
df.reset_index(inplace=True)
df['total'] = df.overview + ' ' + df.cast3 + ' ' + df.director
df.total[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. TomHanks TimAllen DonRickles JohnLasseter"

In [22]:
# 감독, 주연배우 강조
df['total'] = df.overview + (' ' + df.cast3) * 2 + (' ' + df.director) * 3
df.total[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. TomHanks TimAllen DonRickles TomHanks TimAllen DonRickles JohnLasseter JohnLasseter JohnLasseter"

In [23]:
# 데이터는 20000 개로 제한
df = df.head(20000)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.total)
total_tv.shape

(20000, 76759)

In [25]:
# 검색을 위한 테이블
indices = pd.Series(df.index, index=df.title)

#### 3. 코사인 유사도

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim_total = cosine_similarity(total_tv)

In [27]:
def get_recommendation(title, cos_sim=cos_sim_total):
    index = indices[title]
    sim_scores = pd.Series(cos_sim[index])
    movie_indices = sim_scores.sort_values(ascending=False).head(11).tail(10).index
    return df.title[movie_indices]

In [28]:
get_recommendation('Toy Story')

2945              Toy Story 2
14709             Toy Story 3
18141                 Tin Toy
18235             Knick Knack
10325                Luxo Jr.
18191             Red's Dream
16493            Larry Crowne
16552                  Cars 2
15465    Crazy on the Outside
2212             A Bug's Life
Name: title, dtype: object